In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers import initializers
from tensorflow.examples.tutorials.mnist import input_data
from math import sqrt

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.0001
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = './tensorflow_logs/mnist'
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 28*28 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [4]:
x = tf.placeholder(tf.float32, shape=[None, n_input], name='InputData')
y = tf.placeholder(tf.float32, shape=[None, n_classes], name='LabelData')
x_image = tf.reshape(x, [-1,28,28,1], name='image')

In [5]:
def NN(inputs):
    with slim.arg_scope([slim.fully_connected, slim.conv2d], 
                        activation_fn=tf.nn.relu,
                        weights_initializer=initializers.xavier_initializer(),
                        normalizer_fn=slim.batch_norm):
        out = slim.conv2d(inputs, 32, [3, 3])
        print (out.get_shape())
        out = slim.conv2d(inputs, 64, [3, 3])
        print (out.get_shape())
        out = slim.flatten(out)
        print (out.get_shape())
        out = slim.fully_connected(out, 256)
        print (out.get_shape())
        out = slim.fully_connected(out, 256)
        print (out.get_shape())
        out = slim.fully_connected(out, 10, activation_fn=None, normalizer_fn=None)
        print (out.get_shape())
    return out

In [6]:
with tf.name_scope('Model'):
    # Model
    pred = NN(x_image)
with tf.name_scope('Cost'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

(?, 28, 28, 32)
(?, 28, 28, 64)
(?, 50176)
(?, 256)
(?, 256)
(?, 10)


In [7]:
pred.get_shape()

TensorShape([Dimension(None), Dimension(10)])

In [8]:
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.scalar_summary("cost", cost)
# Create a summary to monitor accuracy tensor
tf.scalar_summary("accuracy", accuracy)

# Merge all summaries into a single op
merged_summary_op = tf.merge_all_summaries()

In [9]:
%%time

# images, labels = mnist.train.next_batch(batch_size)
# predictions = NN()

    

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

#     # op to write logs to Tensorboard
    summary_writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, acc, summary = sess.run([optimizer, cost, accuracy, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print ("Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost), "acc =", "{:.9f}".format(acc))

    print ("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print ("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

    print ("Run the command line:\n" \
          "--> tensorboard --logdir=./tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Epoch: 0001 cost = 0.171311664 acc = 0.979999959
Epoch: 0002 cost = 0.054539816 acc = 0.989999950
Epoch: 0003 cost = 0.026153095 acc = 0.990000010
Epoch: 0004 cost = 0.015361330 acc = 0.989999950
Epoch: 0005 cost = 0.010685788 acc = 0.999999940
Epoch: 0006 cost = 0.007054420 acc = 1.000000000
Epoch: 0007 cost = 0.005947883 acc = 0.999999940
Epoch: 0008 cost = 0.007906051 acc = 1.000000000
Epoch: 0009 cost = 0.004764625 acc = 0.999999940
Epoch: 0010 cost = 0.005902671 acc = 1.000000000
Epoch: 0011 cost = 0.003406597 acc = 1.000000000
Epoch: 0012 cost = 0.002238288 acc = 0.999999940
Epoch: 0013 cost = 0.003214744 acc = 0.999999940
Epoch: 0014 cost = 0.005980855 acc = 1.000000000
Epoch: 0015 cost = 0.003139885 acc = 1.000000000
Epoch: 0016 cost = 0.001174563 acc = 1.000000000
Epoch: 0017 cost = 0.000880718 acc = 0.999999940
Epoch: 0018 cost = 0.000925447 acc = 1.000000000
Epoch: 0019 cost = 0.002611917 acc = 0.999999940
Epoch: 0020 cost = 0.005990791 acc = 0.999999940
Epoch: 0021 cost = 0

ResourceExhaustedError: OOM when allocating tensor with shape[10000,64,28,28]
	 [[Node: Model/Conv_1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](image, Conv_1/weights/read)]]
	 [[Node: Model/fully_connected_1/BatchNorm/moments/sufficient_statistics/Shape/_43 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_22_Model/fully_connected_1/BatchNorm/moments/sufficient_statistics/Shape", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Model/Conv_1/convolution', defined at:
  File "c:\users\ikryukov\appdata\local\programs\python\python35\lib\runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\ikryukov\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _ru